***
    
Demo Tensorflow for PINN (Physics-Informed Neural Network)
    
***

    Version: 2023-02-24
    
    Author : Nicholas Sung Wei Yong, Wong Jian Cheng, Ooi Chin Chun, Abhishek Gupta

    Contain:
    
    1. PINN for solving Korteweg–De Vries equation 
    
    2. Optimize PINN using CMA-ES on the Tensorflow framework
       -  as described in Nicholas Sung Wei Yong, Jian Cheng Wong, Pao-Hsiung Chiu, Abhishek Gupta, Chinchun Ooi, Yew-Soon Ong
       "Neuroevolution Surpasses Stochastic Gradient Descent for Physics-Informed Neural Networks" arXiv preprint arXiv:2212.07624 (2022).

Import Libraries

In [1]:
# !pip install pyDOE

In [2]:
%matplotlib inline
import matplotlib.pylab as plt 
from matplotlib import cm
import cma
import os
import math
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint, Callback
import tensorflow.keras.backend as K
# from pyDOE import lhs
from IPython.display import clear_output
from time import time, gmtime, strftime, localtime

import warnings
warnings.filterwarnings("ignore")

2022-12-13 17:01:01.060651: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
from scipy import dot, exp, log, sqrt, floor, ones, randn, zeros_like, Inf, argmax, argmin, eye, outer, zeros
from scipy import array, power, diag, cos, sin, pi
from scipy.linalg import expm, cholesky, solve, norm, eig
from scipy.stats import multivariate_normal

import seaborn as sns

## run on CPU...
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# disable eager excution
tf.compat.v1.disable_eager_execution()
!nvidia-smi --query-gpu=name --format=csv,noheader

NVIDIA GeForce RTX 3090
NVIDIA GeForce RTX 3090




### 1. PINN for kdv equation

#### 1.0. Data

#### 1.1. Geometry, PDE parameter & BC

In [4]:
v1 = 1
v2 = 0.001
# initial condition
c1, c2, x1, x2 = 0.3, 0.1, 0.4, 0.8
a1, a2 = 0.5*np.sqrt(c1/v2), 0.5*np.sqrt(c2/v2)

# spatial domain
x_l, x_u = 0., 1.5

# time domain: 0 - t_T
t_T = 2.0  

t_fitness = 0

In [5]:
# function to generate analytical solution of IC
def f_ic(x):
    return 3*c1*(1 / np.cosh(a1*(x-x1)))**2 + 3*c2*(1 / np.cosh(a2*(x-x2)))**2

#### 1.2. Sampling plan

In [6]:
sim = pd.read_csv('kdv.csv')
# sim = pd.read_csv('/home/sungnwy/kdv plot/kdv.csv')
sim['x'], sim['t'] = sim['x'], sim['t']
x_train = np.vstack([sim.x.values, sim.t.values]).T
y_train = sim[['u']].values

x = x_train[:,0:1].reshape(-1)
t = x_train[:,1:2].reshape(-1)
_train1 = np.argwhere(x <= 1.5)
x_train1 = x[_train1].reshape(-1)
t_train1 = t[_train1].reshape(-1)
y_train1 = y_train[_train1].reshape(-1)
_train2 = np.argwhere(x_train1 >= 0.0)
x_train2 = x_train1[_train2].reshape(-1,1)
t_train2 = t_train1[_train2].reshape(-1,1)
y_train2 = y_train1[_train2].reshape(-1,1)
batch_X = np.hstack([t_train2, x_train2])

#### 1.3. PDE-NN

In [7]:
# specify NN model + Physis 
def create_nn(nodes):

    # input layers -> split into (t, x, z)
    inputs = layers.Input(shape=(2,))
    t, x= layers.Lambda( lambda k: tf.split(k, num_or_size_splits=2, axis=1))(inputs)

    # hidden layers
    hidden_1 = layers.Dense(nodes, activation='tanh', use_bias=False)(layers.Concatenate()([t, x]))
    hidden_2 = layers.Dense(nodes, activation='tanh')(hidden_1)
    hidden_3 = layers.Dense(nodes, activation='tanh')(hidden_2)
    hidden_l = layers.Dense(nodes, activation='tanh')(hidden_3)

    # output layers 
    u = layers.Dense(1, name="u", use_bias=False)(hidden_l)

    # axillary PDE outputs
    u_t = K.gradients(u, t)[0]
    u_x = K.gradients(u, x)[0]
    u_xx = K.gradients(u_x, x)[0]
    u_xxx = K.gradients(u_xx, x)[0]
    pde_residuals = u_t + v1*u*u_x + v2*u_xxx

    # PDE residuals (u_t + v1*u*u_x + v2*u_xxx = 0)
    pde_mse = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(pde_residuals), predictions=pde_residuals)

    # initial conditions: u(0, x) = 3*c1*sech2(a1(x-x1)) + 3*c2*sech2(a2(x-x2))
    x_0, u_0 = tf.boolean_mask(x, tf.equal(t, 0)), tf.boolean_mask(u, tf.equal(t, 0))
    ic = 3*c1*(1 / tf.math.cosh(a1*(x_0-x1)))**2 + 3*c2*(1 / tf.math.cosh(a2*(x_0-x2)))**2
    ic_mse = tf.compat.v1.losses.mean_squared_error (labels=ic, predictions=u_0)


    # initiate model
    nn = models.Model(inputs=inputs, outputs=u)

    # optimizer
    optimizer = tf.keras.optimizers.Adam(0.05)

    # compile model with [physics] loss 
    nn.compile(loss = compute_physic_loss(pde_mse, ic_mse),
               optimizer = optimizer,
               metrics = [compute_pde_loss(pde_mse), compute_ic_loss(ic_mse)])

    # pathway to variables inside NN
    insiders = [u, pde_mse, ic_mse]
    eval_ins = K.function([nn.input, K.learning_phase()], insiders)   # evaluation function

    return (nn, eval_ins)


In [8]:
# define loss function (PDE + IC loss)
def compute_physic_loss(pde_mse, ic_mse):
    def physic_loss(y_true, y_pred):
        pde_loss  = pde_mse # PDE
        ic_loss   = ic_mse  # IC
        return pde_loss + ic_loss
    return physic_loss

# define loss function (PDE loss)
def compute_pde_loss(pde_mse):
    def pde_loss(y_true, y_pred): return pde_mse
    return pde_loss # return a function

# define loss function (IC loss)
def compute_ic_loss(ic_mse):
    def ic_loss(y_true, y_pred): return ic_mse
    return ic_loss # return a function

# define loss function (data loss)
def compute_data_loss(z):
    def data_loss(y_true, y_pred):
        p_data = tf.equal(z, 1)
        return tf.losses.mean_squared_error(labels=tf.boolean_mask(y_true, p_data), predictions=tf.boolean_mask(y_pred, p_data))
    return data_loss

### 3. Optimize PINN with [Neuroevolution / xNES]

In [9]:
# initiate NN model (& pathway to internal values)
n_nodes = 8
nn, eval_ins = create_nn(n_nodes)

# initial weights
w0 = np.array([])
nn_weights = nn.get_weights()
for _g in nn_weights: w0 = np.append(w0, _g.flatten())

nn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 [(None, 1), (None, 1 0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 2)            0           lambda[0][0]                     
                                                                 lambda[0][1]                     
__________________________________________________________________________________________________
dense (Dense)                   (None, 8)            16          concatenate[0][0]            

2022-12-13 17:01:04.510801: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-12-13 17:01:04.553924: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-13 17:01:04.553963: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ihpcgs-01
2022-12-13 17:01:04.553969: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ihpcgs-01
2022-12-13 17:01:04.554132: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.141.3
2022-12-13 17:01:04.554159: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.141.3
2022-12-13 17:01:04.554164: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.141.3
2022-12-13 17:01:04.554674: I tensorflow/core/platform/cpu_fe

In [10]:
# nn weights structure 
nn_weights = nn.get_weights()
nn_wlen = len(nn_weights)
nn_wshape = [_w.shape for _w in nn_weights]
nn_widx = np.cumsum([len(_w.flatten()) for _w in nn_weights])
nn_nweights = nn_widx[-1]

In [11]:
def nn_fitness(_weights):
    _weights = np.split(_weights, nn_widx[:-1])
    _weights = [_weights[i].reshape(nn_wshape[i]) for i in range(nn_wlen)]
    nn.set_weights(_weights)
    _x = batch_X
    _, pde_mse, ic_mse = eval_ins(_x)
    fitness = (pde_mse + ic_mse)
    return fitness

In [13]:
x, es = cma.fmin2(nn_fitness, w0, 5e-2, options={'ftarget':1e-04 ,'popsize':50, 'maxiter':350, 'verb_disp':1})

(25_w,50)-aCMA-ES (mu_w=14.0,w_1=14%) in dimension 240 (seed=1079773, Tue Dec 13 17:10:24 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     50 1.380307674407959e-01 1.0e+00 4.82e-02  5e-02  5e-02 0:00.5
    2    100 1.305103898048401e-01 1.0e+00 4.68e-02  5e-02  5e-02 0:01.1
    3    150 1.073689088225365e-01 1.0e+00 4.57e-02  5e-02  5e-02 0:01.6
    4    200 5.947285890579224e-02 1.0e+00 4.49e-02  4e-02  4e-02 0:02.2
    5    250 5.757907778024673e-02 1.0e+00 4.44e-02  4e-02  4e-02 0:02.7
    6    300 5.567182227969170e-02 1.0e+00 4.40e-02  4e-02  4e-02 0:03.3
    7    350 5.676807090640068e-02 1.0e+00 4.37e-02  4e-02  4e-02 0:03.8
    8    400 5.507174879312515e-02 1.0e+00 4.34e-02  4e-02  4e-02 0:04.4
    9    450 5.403392761945724e-02 1.0e+00 4.31e-02  4e-02  4e-02 0:04.9
   10    500 5.350517109036446e-02 1.0e+00 4.28e-02  4e-02  4e-02 0:05.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
   11    550 5.327090993523598

  102   5100 2.582370303571224e-02 1.3e+00 5.30e-02  5e-02  5e-02 0:55.1
  103   5150 2.528420463204384e-02 1.3e+00 5.33e-02  5e-02  5e-02 0:55.7
  104   5200 2.400267310440540e-02 1.3e+00 5.36e-02  5e-02  5e-02 0:56.2
  105   5250 2.642455324530602e-02 1.3e+00 5.39e-02  5e-02  5e-02 0:56.7
  106   5300 2.523511275649071e-02 1.3e+00 5.41e-02  5e-02  5e-02 0:57.3
  107   5350 2.501249127089977e-02 1.3e+00 5.44e-02  5e-02  6e-02 0:57.9
  108   5400 2.452723868191242e-02 1.3e+00 5.44e-02  5e-02  6e-02 0:58.4
  109   5450 2.468856051564217e-02 1.3e+00 5.46e-02  5e-02  6e-02 0:58.9
  110   5500 2.614720538258553e-02 1.3e+00 5.47e-02  5e-02  6e-02 0:59.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
  111   5550 2.527030557394028e-02 1.3e+00 5.49e-02  5e-02  6e-02 1:00.0
  112   5600 2.610958740115166e-02 1.3e+00 5.50e-02  5e-02  6e-02 1:00.6
  113   5650 2.592662908136845e-02 1.3e+00 5.53e-02  5e-02  6e-02 1:01.1
  114   5700 2.718753740191460e-02 1.3e+00 5.56e-02 

  205  10250 1.981849782168865e-02 1.5e+00 6.35e-02  6e-02  6e-02 1:50.7
  206  10300 2.036021463572979e-02 1.5e+00 6.38e-02  6e-02  6e-02 1:51.2
  207  10350 2.080755494534969e-02 1.5e+00 6.42e-02  6e-02  7e-02 1:51.8
  208  10400 2.071472257375717e-02 1.5e+00 6.46e-02  6e-02  7e-02 1:52.3
  209  10450 2.018992789089680e-02 1.5e+00 6.50e-02  6e-02  7e-02 1:52.8
  210  10500 2.018767967820168e-02 1.5e+00 6.54e-02  6e-02  7e-02 1:53.3
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
  211  10550 2.241992950439453e-02 1.5e+00 6.58e-02  6e-02  7e-02 1:53.9
  212  10600 2.037903666496277e-02 1.5e+00 6.61e-02  7e-02  7e-02 1:54.4
  213  10650 2.079105190932751e-02 1.5e+00 6.64e-02  7e-02  7e-02 1:54.9
  214  10700 2.138561382889748e-02 1.5e+00 6.66e-02  7e-02  7e-02 1:55.4
  215  10750 1.874079927802086e-02 1.5e+00 6.69e-02  7e-02  7e-02 1:56.0
  216  10800 2.252891100943089e-02 1.5e+00 6.71e-02  7e-02  7e-02 1:56.5
  217  10850 2.138701081275940e-02 1.5e+00 6.74e-02 

  308  15400 1.820851676166058e-02 1.6e+00 6.27e-02  6e-02  6e-02 2:46.4
  309  15450 1.856572180986404e-02 1.6e+00 6.25e-02  6e-02  6e-02 2:46.9
  310  15500 1.876136474311352e-02 1.6e+00 6.23e-02  6e-02  6e-02 2:47.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
  311  15550 1.964604482054710e-02 1.6e+00 6.23e-02  6e-02  6e-02 2:48.0
  312  15600 2.041749656200409e-02 1.6e+00 6.22e-02  6e-02  6e-02 2:48.5
  313  15650 1.886617019772530e-02 1.6e+00 6.21e-02  6e-02  6e-02 2:49.1
  314  15700 1.700617000460625e-02 1.6e+00 6.20e-02  6e-02  6e-02 2:49.6
  315  15750 1.592314243316650e-02 1.6e+00 6.18e-02  6e-02  6e-02 2:50.1
  316  15800 1.875044405460358e-02 1.6e+00 6.16e-02  6e-02  6e-02 2:50.7
  317  15850 1.737122423946857e-02 1.6e+00 6.14e-02  6e-02  6e-02 2:51.2
  318  15900 1.517164893448353e-02 1.6e+00 6.13e-02  6e-02  6e-02 2:51.8
  319  15950 1.747895032167435e-02 1.6e+00 6.11e-02  6e-02  6e-02 2:52.3
  320  16000 1.718255877494812e-02 1.6e+00 6.10e-02 